# Treponema genome analysis workflow - Grillova et al. 2019

## The environment

### Tested software versions and operation system

The workflows has been tested on Linux machine (Ubuntu 16.04) with Python 2.7.6, Python 3.4.3 using Conda 4.5.11, Jupyter notebook 5.7.2 and bash_kernel 0.7.1.

In [ ]:
uname -a
lsb_release -a
#export PATH=/mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2/bin:$PATH # In case we use different version of Conda than the system-wide installation
which python3
python3 --version
which conda
conda --version
echo "Jupyter notebook" `jupyter notebook --version`
echo "bash_kernel" `pip3 show bash_kernel | grep Version`

## The analysis

First, we can activate the environment and check 

In [1]:
conda activate treponema
# In case we want to export the system settings and software versions
conda info
conda list
# In case we modified the environment want to export it
#conda env export > treponema_mod.yml

(treponema) (treponema) 
     active environment : treponema
    active env location : /mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2/envs/treponema
            shell level : 2
       user config file : /mnt/nfs/home/325073/000000-My_Documents/VM-home/.condarc
 populated config files : /mnt/nfs/home/325073/000000-My_Documents/VM-home/.condarc
          conda version : 4.6.14
    conda-build version : not installed
         python version : 2.7.16.final.0
       base environment : /mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2  (read only)
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo.anaconda.com/pkgs/fre

: 1

## Running the workflow

### Input variables

First, we have to setup few variables which will be used throughout the analysis.

In [ ]:
INPUT_DIR="/mnt/nfs/home/323639/000000-My_Documents/VM-home/projects/honza/linda/pipeline/data/raw" # Directory with the raw paired input files in fastq.gz - first pair has to be named xxx_R1.fastq.gz and the second xxx_R2.fastq.gz otherwise you would have to change the input suffixes for the individual steps
OUTPUT_DIR="/mnt/nfs/home/323639/000000-My_Documents/VM-home/projects/honza/linda/pipeline/results" # Directory where we want to save the results

REFERENCE="/mnt/nfs/home/323639/000000-My_Documents/VM-home/projects/honza/linda/pipeline/data/references/SS14.fa.gz" # Bacteria reference genome
HOST_GENOME="/mnt/nfs/home/323639/000000-My_Documents/VM-home/projects/honza/linda/pipeline/data/references/GCF_000001405.36_GRCh38.p10_genomic.fna.gz" # Host genome reference sequence; for human - ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.36_GRCh38.p10/GCF_000001405.36_GRCh38.p10_genomic.fna.gz

THREADS=12 # Number of threads we will use in the analysis

ADAPTER_R1="CTGTCTCTTATACACATCT" # R1 3' adapeter, if any
ADAPTER_R2=$ADAPTER_R1 # R2 3' adapter, if any

### Initial quality check

It is always a good idea to run an initial quality check on your data.

In [ ]:
mkdir -p $OUTPUT_DIR/qc/fastqc/raw

ls $INPUT_DIR/*.gz

fastqc --threads $THREADS --outdir $OUTPUT_DIR/qc/fastqc/raw $INPUT_DIR/*.gz

multiqc --outdir $OUTPUT_DIR/qc/fastqc/raw $OUTPUT_DIR/qc/fastqc/raw/ 

### Read preprocessing

Since we are working with DNA data and aiming for the results including the polymorphisms we should perform a careful preprocessing to remove the adapter sequences and perform quality trimming.

In [ ]:
mkdir -p $OUTPUT_DIR/data/preprocessed
mkdir $OUTPUT_DIR/qc/cutadapt
mkdir $OUTPUT_DIR/qc/fastqc/preprocessed

cd $INPUT_DIR/

for sample in *R1*.gz
do
    FORWARD=$sample
    extension="${FORWARD##*R1}"
    REVERSE=${FORWARD%R1*}R2${extension}

    # Input file check
    if [ ! -f ${FORWARD} ]; then
    echo "Input file not found! First in pair."; echo ${FORWARD}
    fi
    if [ ! -f ${REVERSE} ]; then
    echo "Input file not found! Second in pair."; echo ${REVERSE}
    fi

    echo "Now I am processing ${FORWARD} as first in a pair and ${REVERSE} as a second in a pair."

    cutadapt -a $ADAPTER_R1 -A $ADAPTER_R2 \
    --times 1 --quality-cutoff 15,15 --trim-n \
    --error-rate 0.10 -O 3 --minimum-length 35 --max-n 0 \
    --output $OUTPUT_DIR/data/preprocessed/${FORWARD%$extension}.trimmed.fastq.gz \
    --paired-output $OUTPUT_DIR/data/preprocessed/${REVERSE%$extension}.trimmed.fastq.gz \
    $FORWARD $REVERSE &>$OUTPUT_DIR/qc/cutadapt/${FORWARD%_R1${extension}}.cutadapt.out

    echo "Done processing $FORWARD and $REVERSE"
done

multiqc --outdir $OUTPUT_DIR/qc/cutadapt $OUTPUT_DIR/qc/cutadapt/

fastqc --threads $THREADS --outdir $OUTPUT_DIR/qc/fastqc/preprocessed $OUTPUT_DIR/data/preprocessed/*.gz

multiqc --outdir $OUTPUT_DIR/qc/fastqc/preprocessed $OUTPUT_DIR/qc/fastqc/preprocessed/

### Host genome contamination removal

Removal of the host genome DNA before the analysis speeds up the analysis and we will be working with much smaller files as well. 

As is usuall, we have to generate a host genome DNA reference index. This is run just once for one reference and one BBMap version.

In [ ]:
mkdir $(dirname $HOST_GENOME)/bbmap_index
echo $(dirname $HOST_GENOME)/bbmap_index

# If the host genome index does not exist create it
if [ ! -d "$(dirname $HOST_GENOME)/bbmap_index/$(basename $HOST_GENOME)" ]; then
    mkdir $(dirname $HOST_GENOME)/bbmap_index/$(basename $HOST_GENOME)
    bbmap.sh ref=$HOST_GENOME -Xmx20g path=$(dirname $HOST_GENOME)/bbmap_index/$(basename $HOST_GENOME)
fi

Once the index is done we can launch the host-genome removal.

In [ ]:
mkdir $OUTPUT_DIR/qc/bbmap

cd $OUTPUT_DIR/data/preprocessed/

for sample in *R1*trimmed.fastq.gz
do
    FORWARD=$sample
    extension="${FORWARD##*R1}"
    REVERSE=${FORWARD%R1*}R2${extension}

    # Input file check
    if [ ! -f ${FORWARD} ]; then
        echo "Input file not found! First in pair."; echo ${FORWARD}
    fi
    if [ ! -f ${REVERSE} ]; then
        echo "Input file not found! Second in pair."; echo ${REVERSE}
    fi

    echo "Now I am processing ${FORWARD} as first in a pair and ${REVERSE} as a second in a pair with reference $HOST_GENOME."

    # Start mapping
    bbmap.sh threads=$THREADS -Xmx25g minid=0.95 maxindel=3 bandwidthratio=0.16 \
    bandwidth=12 quickmatch fast minhits=2 path=$(dirname $HOST_GENOME)/bbmap_index/$(basename $HOST_GENOME) unpigz pigz \
    in=${FORWARD} in2=${REVERSE} outu=$OUTPUT_DIR/data/preprocessed/${FORWARD%_R1${extension}}.clean.fastq.gz \
    outm=$OUTPUT_DIR/data/preprocessed/${FORWARD%_R1${extension}}.dirty.fastq.gz &>$OUTPUT_DIR/qc/bbmap/${FORWARD%_R1${extension}}.bbmap.out # qtrim=rl trimq=10 untrim  # We already have preprocessed data, no need for this

    # De-interleave
    reformat.sh -Xmx12g verifypaired in=$OUTPUT_DIR/data/preprocessed/${FORWARD%_R1${extension}}.clean.fastq.gz \
    out1=$OUTPUT_DIR/data/preprocessed/${FORWARD%${extension}}.clean.fastq.gz out2=$OUTPUT_DIR/data/preprocessed/${REVERSE%${extension}}.clean.fastq.gz

    # Remove the host genome mapped reads (usefull for mapping precision)
    rm $OUTPUT_DIR/data/preprocessed/${FORWARD%_R1${extension}}.clean.fastq.gz
    rm $OUTPUT_DIR/data/preprocessed/${FORWARD%_R1${extension}}.dirty.fastq.gz
done

mkdir $OUTPUT_DIR/qc/fastqc/clean

fastqc --threads $THREADS --outdir $OUTPUT_DIR/qc/fastqc/clean $OUTPUT_DIR/data/preprocessed/*.clean.fastq.gz

multiqc --outdir $OUTPUT_DIR/qc/fastqc/clean $OUTPUT_DIR/qc/fastqc/clean/

### Bacteria contamination scan

Before we start with the alignment we can quickly scan for possible bacterial contamination in our dataset. This scan uses default StrainSeeker database which is most likely outdated but StrainSeeker offers a possibility to generate your [own index](http://bioinfo.ut.ee/strainseeker/index.php?r=site/page&view=manual#database) for the scan with their builder script. One advantage over tools such as [Kraken2](https://ccb.jhu.edu/software/kraken2/) (a great tool) is that is consumes much less RAM. However, the latest releases of MiniKraken2 could be used as well.  

Before we can use StrainSeeker we have to download the database to scan. We can use the database provided at the StrainSeeker webpage or create our own. 

In [ ]:
STRAINSEEKER_DB=/mnt/nfs/home/323639/000000-My_Documents/VM-home/projects/honza/linda/pipeline/data/references/strainseekerdb

mkdir $STRAINSEEKER_DB
cd $STRAINSEEKER_DB/
wget http://bioinfo.ut.ee/strainseeker/executables/ss_db_w32_4324.tar.gz
tar xvzf ss_db_w32_4324.tar.gz

STRAINSEEKER_DB=$STRAINSEEKER_DB/ss_db_w32_4324

Once we have the database we can start the scan. 

In [ ]:
mkdir $OUTPUT_DIR/qc/strainseeker

cd $OUTPUT_DIR/data/preprocessed/

for sample in *.clean.fastq.gz
do 
    echo "Working on sample $sample"

    echo "Subsampling"
    seqtk sample -s100 $sample 1000000 > $sample.sub # Subsample fastq to 1M

    echo "Scanning"
    seeker.pl -i $sample.sub -d $STRAINSEEKER_DB -o $OUTPUT_DIR/qc/strainseeker/${sample%.fastq.gz}.seeker.txt

    rm $sample.sub
done

### Bacteria contamination scan - Kraken2 (optional)

If you have enough resources you can use [Kraken2](https://ccb.jhu.edu/software/kraken2/). It scans the most recent bacterial, viral and fungal databases (or their subselection) and evaluates the possible distributions of individual species. If you decided to use Kraken2, you have to build the Kraken2 database. Please note this might take a while and will consume quite a lof of [RAM](https://ccb.jhu.edu/software/kraken2/index.shtml?t=manual#kraken-2-databases). 

In [ ]:
KRAKEN2_DB="/mnt/nfs/home/323639/000000-My_Documents/VM-home/projects/honza/linda/pipeline/data/references/kraken2db" # Directory with stored StrainSeeker database
mkdir $KRAKEN2_DB

DATE=$(date +'%m%d%Y') # Save current date -> database version
echo $DATE

kraken2-build --standard --threads $THREADS --db $KRAKEN2_DB/$DATE

KRAKEN2_DB=$KRAKEN2_DB/06182019

Once we have the database we can start the scan. 

In [ ]:
THREADS=12
OUTPUT_DIR=/mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/results
KRAKEN2_DB=/mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/reference/kraken2db/06182019
REFERENCE=/mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/reference/SS14.fa.gz

In [ ]:
mkdir -p $OUTPUT_DIR/qc/kraken2/reads

cd $OUTPUT_DIR/data/preprocessed/

for sample in *R1*.clean.fastq.gz
do
    FORWARD=$sample
    extension="${FORWARD##*R1}"
    REVERSE=${FORWARD%R1*}R2${extension}

    # Input file check
    if [ ! -f ${FORWARD} ]; then
        echo "Input file not found! First in pair."; echo ${FORWARD}
    fi
    if [ ! -f ${REVERSE} ]; then
        echo "Input file not found! Second in pair."; echo ${REVERSE}
    fi

    echo "Now I am processing ${FORWARD} as first in a pair and ${REVERSE} as a second in a pair with database $KRAKEN2_DB."
    
    kraken2 --use-names --paired --threads $THREADS --gzip-compressed --classified-out $OUTPUT_DIR/qc/kraken2/reads/${FORWARD%R1*}.classified-out#.fastq --unclassified-out $OUTPUT_DIR/qc/kraken2/reads/${FORWARD%R1*}.unclassified-out#.fastq --output $OUTPUT_DIR/qc/kraken2/reads/${FORWARD%R1*}.kraken.txt --report $OUTPUT_DIR/qc/kraken2/reads/${FORWARD%R1*}.kraken.report.txt --db $KRAKEN2_DB $FORWARD $REVERSE

    for output in $OUTPUT_DIR/qc/kraken2/reads/*classified*.fastq
    do
        gzip $output &
    done 
done

wait

# Get only species lines
for report in $OUTPUT_DIR/qc/kraken2/reads/*.kraken.report.txt
do 
    grep -P 'unclassified|root|\tS\t' $report | sort -k1,1nr > ${report%.kraken.report.txt*}.S.kraken.report.txt
done

### Bacteria reference genome alignment

With host genome DNA cleaned data we can proceed to the alignment to the reference. Please note we apply few filterings already at this step, mainly the minimal mapping quality, pairing of the reads and read duplication.

In [ ]:
# Prepare reference indexes
if [[ ${REFERENCE##*.} == "gz" ]] # Uncompress the reference if it is in gz archive
then
    gunzip -c $REFERENCE > ${REFERENCE%.gz*}
    REFERENCE=${REFERENCE%.gz*}
fi

if [[ `echo $REFERENCE | grep ".fna$"` != "" ]] || [[ `echo $REFERENCE | grep ".fasta$"` != "" ]] # Make a link to the original to have ".fa" suffix
then
    ln -s $REFERENCE ${REFERENCE%.*}.fa
fi

bwa index $REFERENCE
samtools faidx $REFERENCE

cd $OUTPUT_DIR/data/preprocessed

OUTPUT_DIR=${OUTPUT_DIR}/$(basename $REFERENCE .fa} # Adjust output directory to reflect the used reference genome

mkdir -p $OUTPUT_DIR/alignment
mkdir -p $OUTPUT_DIR/qc/alignment_stats

for sample in *R1*clean.fastq.gz
do
    FORWARD=$sample
    extension="${FORWARD##*R1}"
    REVERSE=${FORWARD%R1*}R2${extension}

    # Input file check
    if [ ! -f ${FORWARD} ]; then
        echo "Input file not found! First in pair."; echo ${FORWARD}
    fi
    if [ ! -f ${REVERSE} ]; then
        echo "Input file not found! Second in pair."; echo ${REVERSE}
    fi

    # Start mapping
    echo "Now I am processing ${FORWARD} as first in a pair and ${REVERSE} as a second in a pair with reference $REFERENCE"

    bwa mem -t $THREADS -T 20 -v 1 -M -R "@RG\tID:1\tLB:${sample%%.*}\tPL:Illumina\tSM:${sample%%.*}\tPU:${sample%%.*}" $REFERENCE ${FORWARD} ${REVERSE} | samtools view -F 4 -@ $THREADS -b - | samtools sort -@ $THREADS - > $OUTPUT_DIR/alignment/${FORWARD%R1*}.$(basename $REFERENCE .fa).bam 

    echo "Mapping finished"

    echo "Starting post-alignment processing and basic filtering"
    
    cd $OUTPUT_DIR/alignment

    i=${FORWARD%R1*}.$(basename $REFERENCE .fa).bam

    samtools index -@ $THREADS $i # Index BAM files
    samtools flagstat $i > $OUTPUT_DIR/qc/alignment_stats/${i%.*}.flagstat &
    samtools view -@ $THREADS -h -F 12 -f 2 -F 256 -b $i | samtools sort -n -@ $THREADS - | samtools fixmate -O bam - - | samtools sort -@ $THREADS - > ${i%.*}.filt.bam # -F 2048 = supplementary alignment, "chimeric/non-linear alignments"; -q $MAPQ
    samtools index -@ $THREADS ${i%.*}.filt.bam
    samtools flagstat ${i%.*}.filt.bam > $OUTPUT_DIR/qc/alignment_stats/${i%.*}.filt.flagstat

    echo "Filtering finished"

    echo "Starting indel realignment"
    # Indel realignment
    # Create input seqence dictionary and index reference
    rm ${REFERENCE%.*}.dict
    picard CreateSequenceDictionary R=$REFERENCE O=${REFERENCE%.*}.dict

    # Realign
    i=${i%.*}.filt.bam

    gatk -T RealignerTargetCreator --num_threads $THREADS -R $REFERENCE -I ${i} -o $OUTPUT_DIR/alignment/${i%.*}.forIndelRealigner.intervals # Prepare intervals
    gatk -I ${i} -R $REFERENCE -T IndelRealigner -LOD 2.5 --consensusDeterminationModel USE_SW -targetIntervals $OUTPUT_DIR/alignment/${i%.*}.forIndelRealigner.intervals -o $OUTPUT_DIR/alignment/${i%.*}.indelRealigned.bam # Run re-alignment

    samtools index -@ $THREADS ${i%.*}.indelRealigned.bam

    rm $i
    rm $i*
    rm $OUTPUT_DIR/alignment/${i%.*}.forIndelRealigner.intervals

    echo "Finished indel realignment"

    echo "Starting read duplicate removal"

    # Remove duplicates
    i=${i%.*}.indelRealigned.bam

    mkdir $OUTPUT_DIR/qc/picard_dup
    
    picard MarkDuplicates INPUT=$i OUTPUT=${i%.*}.dedup.bam METRICS_FILE=$OUTPUT_DIR/qc/picard_dup/${i%.*}.dedupStats.txt REMOVE_DUPLICATES=true OPTICAL_DUPLICATE_PIXEL_DISTANCE=100 # Taggs ALL duplicates, PCR and optical and remove them

    samtools index -@ $THREADS ${i%.*}.dedup.bam
    samtools flagstat ${i%.*}.dedup.bam > $OUTPUT_DIR/alignment_stats/${i%.*}.dedup.flagstat

    rm $i
    rm $i*
    
    echo "Finished read duplicate removal"
done

### Post-alignment filtering

The most important part of the processing is the **post-alignment filtering**. `BWA MEM` is known to be very sensitive but not very specific. Right now, we have a lot of alignments which in reality do not belong to our reference. This is due to several factors used in BWA MEM, such as minimal length of alignment (default: 19, can be adjusted by `-k [INT]`), allowance of extensive soft-clipping, allowed hard-clipping and allowing a lot of mismatches. 

The simplest way to see the alignment artifacts and cross-mappings is to check the reference alignment coverage and check highly uneven coverage peaks. 

We apply the following filters:

1. To many mismatches
    * max. 0.05% mismatches and
    * max. 5 mismatches
2. Very short alignments
     * min 35 bp mapping (measured on the read, not the reference)
3. Too much softclipped 
     * max 0.05% soft-clipped
4. Supplementary/chimeric reads 
    * `samtools -F 2048` flag
5. Too much hardclipped
     * max 0.00% hard-clipped (no hard-clipping allowed)
6. MAPQ 10 
    * very often repetitive alignments
7. MAPQ 40 
    * get only high quality alignment
8. Singletons 
    * remove singleton reads (not paired) after all the mapping filtering

In [ ]:
MAX_PERC_OF_MM=0.05 # Maximum percentage of mismatches compared to the read length - bad if we have to error-prone reads but helps to remove false-positives
MAX_NUMBER_OF_MM=5 # Maximum number of mismatches
MIN_LENGTH_MAPPED=35 # Remove mappings that mapped with too few bases (remove excesive soft-clipping)
MAX_SOFTCLIP=0.05 # Maximal percentage of the reads allowed to be soft-clipped
MAX_HARDCLIP=0.00 # Maximal percentage of the reads allowed to be hard-clipped
MAPQ=10 # Minimal MAPQ for (probably) repetitive regions
MAPQ_FINAL=40 # Minimal MAPQ for final results

cd $OUTPUT_DIR/alignment

mkdir $OUTPUT_DIR/tmp

# Filter mappings
for sample in *.filt.indelRealigned.dedup.bam
do 
    echo "Processing sample $sample"

    samtools index -@ $THREADS $sample

    # 1) Remove too many mismatches
    bamutils filter $sample ${sample%.*}.mm1.bam -failed ${sample%.*}.mm1.fail.txt -maximum_mismatch_ratio $MAX_PERC_OF_MM # Filter by perc. of mismatches; It's more filtering on edit distance = how many nucleotides have to be changed to get exactly the reference sequence; indels are counted as many times as they are "long"; error in the source https://github.com/ngsutils/ngsutils/pull/18
    bamutils filter ${sample%.*}.mm1.bam ${sample%.*}.mm2.bam -failed ${sample%.*}.mm2.fail.txt -mismatch $MAX_NUMBER_OF_MM # Filter by num. of mismatches

    cat ${sample%.*}.mm1.fail.txt ${sample%.*}.mm2.fail.txt > ${sample%.*}.mm.fail.txt # Merge mapping filtered by mismatches

    # Use extracted filtered read name and filter it out from the original bam but ONLY when the mismatches filtering had some results
    if [ -s ${sample%.*}.mm.fail.txt ]
    then
        cat ${sample%.*}.mm.fail.txt | awk '{print $1}' | sort -T $OUTPUT_DIR/tmp --parallel=$THREADS | uniq > ${sample%.*}.mm.fail.txt.tmp
        echo "Number of too much mismatched reads is" `wc -l ${sample%.*}.mm.fail.txt.tmp` "for sample " $sample
        picard FilterSamReads I=$sample O=${sample%.*}.mm.filtOut.bam READ_LIST_FILE=${sample%.*}.mm.fail.txt.tmp FILTER=includeReadList # Include reads
        rm ${sample%.*}.mm.fail.txt.tmp
    else
        echo "There are none to much mismatched reads with " $MAX_PERC_OF_MM " % and " $MAX_NUMBER_OF_MM " mismatches for sample " $sample". Nothing to report."
    fi

    # 2) Remove too short alignment
    # Filter out mappings that mapped with length shorter than MIN_LENGTH_MAPPED https://www.biostars.org/p/12406/ - this filters the mapping by calculating the length of mapping on the read itself; example 106S8M1D21M14S with MIN_LENGTH_MAPPED=30 is filtered out because 8M+21M=29
    # Another filtering by mapped read length could be taken from here https://www.biostars.org/p/151510/ - this filters the mapping by calculating the length of mapping on the reference; example 106S8M1D21M14S with MIN_LENGTH_MAPPED=30 is NOT filtered out because 8M+1D+21M=30, it is filtered out when set to MIN_LENGTH_MAPPED=31
    #samjs.jar -e '!record.readUnmappedFlag  && record.cigar.referenceLength  >= $MIN_LENGTH_MAPPED '$sample | samtools view -@ $THREADS -b -o ${sample%.*}.short.bam -
    samtools view -h -@ $THREADS ${sample%.*}.mm2.bam | perl -slane '$l = 0; $F[5] =~ s/(\d+)[MX=DN]/$l+=$1/eg; print if $l < $MIN_LENGTH_MAPPED or /^@/' -- -MIN_LENGTH_MAPPED=$MIN_LENGTH_MAPPED | samtools view -@ $THREADS -b - > ${sample%.*}.short.filtOut.bam &
    samtools view -h -@ $THREADS ${sample%.*}.mm2.bam | perl -slane '$l = 0; $F[5] =~ s/(\d+)[MX=DN]/$l+=$1/eg; print if $l >= $MIN_LENGTH_MAPPED or /^@/' -- -MIN_LENGTH_MAPPED=$MIN_LENGTH_MAPPED | samtools view -@ $THREADS -b - > ${sample%.*}.short.bam

    # 3) Too much softclipped
    # Running it on the whole file and saving it creates error in BAM validation; we have to just take the read names from here and remove them from the alignment
    bamutils removeclipping ${sample%.*}.short.bam ${sample%.*}.scf.bam.tmp
    samtools view -@ $THREADS ${sample%.*}.scf.bam.tmp | grep 'ZC:f:' | awk '{for (i=1;i<=NF;i++){if ($sample ~/ZC:f:/) {print $1, $sample}}}' | sed 's/ZC:f://' | awk -v MAX_SOFTCLIP="$MAX_SOFTCLIP" ' $2 > MAX_SOFTCLIP {print $1}' > ${sample%.*}.read_names_to_remove_highSoftClip.txt.tmp # Get only softclipped reads above the threshold
    rm ${sample%.*}.scf.bam.tmp # Remove temporary BAMs
    cat ${sample%.*}.read_names_to_remove_highSoftClip.txt.tmp | cut -f 1 | sort -T $OUTPUT_DIR/tmp --parallel=$THREADS | uniq > ${sample%.*}.read_names_to_remove_highSoftClip.txt # Get unique read names with softclipping!
    #cat ${sample%.*}.read_names_to_remove_highSoftClip.txt | cut -f 1 | sort -T $OUTPUT_DIR/tmp --parallel=$THREADS | uniq -d > tmp; mv tmp ${sample%.*}.read_names_to_remove_highSoftClip.txt # Get unique read names with softclipping if whole pair failed the filtering!
    rm ${sample%.*}.read_names_to_remove_highSoftClip.txt.tmp

    # Use extracted filtered read name and filter it out from the original bam but ONLY when the soft clipping filtering had some results
    if [ -s ${sample%.*}.read_names_to_remove_highSoftClip.txt ]
    then
        echo "Number of too much soft clipped reads is" `wc -l ${sample%.*}.read_names_to_remove_highSoftClip.txt` "for sample " $sample
        picard FilterSamReads I=${sample%.*}.short.bam O=${sample%.*}.scf.filtOut.bam READ_LIST_FILE=${sample%.*}.read_names_to_remove_highSoftClip.txt FILTER=includeReadList & # Include reads
        picard FilterSamReads I=${sample%.*}.short.bam O=${sample%.*}.scf.bam READ_LIST_FILE=${sample%.*}.read_names_to_remove_highSoftClip.txt FILTER=excludeReadList # Exclude reads
    else
        echo "There are none to much soft clipped reads with " $MAX_SOFTCLIP " % soft clipping for sample " $sample. Continue without filtering.
        mv ${sample%.*}.short.bam ${sample%.*}.scf.bam
    fi

    samtools index -@ $THREADS ${sample%.*}.scf.bam

    # 4) Supplementary/chimeric alignment
    samtools view -@ $THREADS -h -f 2048 -b ${sample%.*}.scf.bam > ${sample%.*}.sup.filtOut.bam & # get -F 2048 "chimeric/non-linear alignments" - might be interesting for translocations; possible overlap with hardclipping 
    samtools view -@ $THREADS -h -F 2048 -b ${sample%.*}.scf.bam > ${sample%.*}.sup.bam # remove -F 2048 "chimeric/non-linear alignments" - might be interesting for translocations; possible overlap with hardclipping

    # 5) Hardclipped
    samtools view -@ $THREADS -h ${sample%.*}.sup.bam | awk '$6 ~ /H/{print}' | samtools view -bh - > ${sample%.*}.hcf.filtOut.bam & # Get only hardclipped mappings; hardclipped alignments might mean the part of the alignment that is hardclipped might map to different part of the genome - possible chimeric reads? https://www.biostars.org/p/109333/
    samtools view -@ $THREADS -h ${sample%.*}.sup.bam | awk '$6 !~ /H/{print}' | samtools view -bh - > ${sample%.*}.hcf.bam # https://www.biostars.org/p/137461/

    # 6) MAPQ10 – very often repetitive alignments
    samtools view -@ $THREADS -h ${sample%.*}.sup.bam | awk -v var="$MAPQ" '$5 < var || $1 ~ /^@/' | samtools view -b - > ${sample%.*}.MAPQ${MAPQ}.filtOut.bam &
    samtools view -@ $THREADS -h -bq $MAPQ ${sample%.*}.sup.bam > ${sample%.*}.MAPQ${MAPQ}.bam # BWA-MEM -T settings should filter all low MAPQ mappings; -F 2048 "chimeric/non-linear alignments" - might be interesting for translocations

    # 7) MAPQ40 – Only high quality alignments
    samtools view -@ $THREADS -h ${sample%.*}.MAPQ${MAPQ}.bam | awk -v var="$MAPQ_FINAL" '$5 < var || $1 ~ /^@/' | samtools view -b - > ${sample%.*}.MAPQ${MAPQ_FINAL}.filtOut.bam &
    samtools view -@ $THREADS -h -bq $MAPQ_FINAL ${sample%.*}.MAPQ${MAPQ}.bam > ${sample%.*}.MAPQ${MAPQ_FINAL}.bam # Get only very high quality mappings

    # 8) Singletons
    # Remove reads that remained as singletons after all filtering steps - works only for paired-end sequencing!
    samtools view -@ $THREADS ${sample%.*}.MAPQ${MAPQ_FINAL}.bam | cut -f 1 | sort -T $OUTPUT_DIR/tmp --parallel=$THREADS | uniq -u > ${sample%.*}.MAPQ${MAPQ_FINAL}.singleAfterFilt.txt

    if [ -s ${sample%.*}.MAPQ${MAPQ_FINAL}.singleAfterFilt.txt ]
    then
        echo "Number of singleton reads after filtering is" `wc -l ${sample%.*}.MAPQ${MAPQ_FINAL}.singleAfterFilt.txt` "for sample " $sample
        picard FilterSamReads I=${sample%.*}.MAPQ${MAPQ_FINAL}.bam O=${sample%.*}.singletons.filtOut.bam READ_LIST_FILE=${sample%.*}.MAPQ${MAPQ_FINAL}.singleAfterFilt.txt FILTER=includeReadList & # Include reads
        picard FilterSamReads I=${sample%.*}.MAPQ${MAPQ_FINAL}.bam O=${sample%.*}.MAPQ${MAPQ_FINAL}.bam.tmp READ_LIST_FILE=${sample%.*}.MAPQ${MAPQ_FINAL}.singleAfterFilt.txt FILTER=excludeReadList # Exclude reads
        mv ${sample%.*}.MAPQ${MAPQ_FINAL}.bam.tmp ${sample%.*}.MAPQ${MAPQ_FINAL}.bam
    else
        echo "There are none singleton reads after filtering for sample " $sample. Continue without filtering.
    fi

    samtools index -@ $THREADS ${sample%.*}.MAPQ${MAPQ_FINAL}.bam

    sleep 60 # wait/sleep for one minute for the jobs to finish (if they didn't)

done

for sample in $OUTPUT_DIR/alignment/*.bam
do
    samtools index -@ $THREADS $sample
done

# Do some cleaning
rm $OUTPUT_DIR/alignment/*.bam.tmp $OUTPUT_DIR/alignment/*.read_names_to_remove_highSoftClip.txt $OUTPUT_DIR/alignment/*.reads $OUTPUT_DIR/alignment/*.singleAfterFilt.txt $OUTPUT_DIR/alignment/*.fail.txt

mkdir -p $OUTPUT_DIR/alignment/filtered/filtOut
mkdir -p $OUTPUT_DIR/alignment/filtered/filt

mv $OUTPUT_DIR/alignment/*.filtOut.bam* $OUTPUT_DIR/alignment/filtered/filtOut/
mv $OUTPUT_DIR/alignment/* $OUTPUT_DIR/alignment/filtered/filt/
mv $OUTPUT_DIR/alignment/filtered/filt/*.MAPQ40.bam* $OUTPUT_DIR/alignment/filtered/

Please note that the filtering applied is rather strict. We might lose some true positive mappings but in this case we rather focus on strongly supported alignments to be sure we are detecting what we want to detect.

### Alignment statistics and reference genome coverage 

One of the the quality checks after the filtering and before all the other steps is the genome coverage statistics...and of course general mapping statistics.

In [ ]:
# Run mapping QC
COVER_INT="-ct 3 -ct 5 -ct 10" # Intervals of coverage to make the statistics for, can be multiple values; --summaryCoverageThreshold

cd $OUTPUT_DIR/alignment/filtered

# Get general alignment statistics
mkdir $OUTPUT_DIR/qc/qualimap

for sample in *.MAPQ${MAPQ_FINAL}.bam
do
    # PDF version is better for browsing
    qualimap bamqc -bam $sample -nt $THREADS -c -outformat PDF -outdir $OUTPUT_DIR/qc/qualimap -outfile ${sample%.*}.qualimap.pdf
    mv $OUTPUT_DIR/qc/qualimap/genome_results.txt $OUTPUT_DIR/qc/qualimap/${sample%.*}.genome_results.txt
    # HTML version is necessary for multiQC
    mkdir -p $OUTPUT_DIR/qc/qualimap/html/${sample%.*}
    qualimap bamqc -bam $sample -nt $THREADS -c -outformat HTML -outdir $OUTPUT_DIR/qc/qualimap/html/${sample%.*} # Has to be redirected to separate folder for each file
done

multiqc -o $OUTPUT_DIR/qc/qualimap $OUTPUT_DIR/qc/qualimap/

# Calculate the coverage
mkdir -p $OUTPUT_DIR/qc/coverage

# Prepare indexes
samtools faidx $REFERENCE
rm ${REFERENCE%.*}.dict
picard CreateSequenceDictionary R=$REFERENCE O=${REFERENCE%.*}.dict

# Run GATK DepthOfCoverage
ls *.MAPQ40.bam | tr ' ' '\n' > $OUTPUT_DIR/alignment/filtered//input_bams.list

echo "Going to process files" `cat $OUTPUT_DIR/alignment/filtered//input_bams.list`

gatk \
-T DepthOfCoverage \
-R $REFERENCE \
-I $OUTPUT_DIR/alignment/filtered/input_bams.list \
-o $OUTPUT_DIR/qc/coverage/$(basename $REFERENCE .fa)${INPUT_BAM%.bam}.coverage \
$COVER_INT # --outputFormat csv # Default is readable table (rtable)

rm $OUTPUT_DIR/alignment/filtered/input_bams.list

pigz -p $THREADS $OUTPUT_DIR/qc/coverage/*.coverage

# Count number of mapped reads - we have the same information in Qualimap report
echo "Number of mapped read pairs (-F 3852 -f 2 -q 40 = read mapped in proper pair ; NOT read unmapped, mate unmapped not primary alignment, read fails platform/vendor quality checks, read is PCR or optical duplicate, supplementary alignment)" > $OUTPUT_DIR/qc/coverage/num_reads.txt

for sample in *.MAPQ40.bam
do
    echo -ne $sample ' '
    samtools view -@ $THREADS -F 3852 -f 2 -q 40 $sample | cut -f 1 | sort -T $OUTPUT_DIR/tmp | uniq | wc -l
done >> $OUTPUT_DIR/qc/coverage/num_reads.txt


### Alignment consensus

In this phase, we can generate the mapping consensus. Please not this steps is closely related to the previous step as it will take only the filtered mappings into consideration. Here, we use a 'simple' variant call performed by `samtools`&`bcftools` and `GATK`. You might consider alternating this step with more sophisticated variant call if you think it is necessary. The difference is that the "samtools" version gives you lower-case letters where coverage was low and *N* where there was no coverage. Here, we convert all low-coverage positions to *n* (lower-case). The "GAKT" version copies the reference sequence where is no coverage. "samtools" version doesn't like to include indels whereas "GATK" version should be OK with it.

#### Fix reference name and SAM header (optional)

`bcftools` and `vcfutils` might have problem with "." or any "strange" symbol in the reference name. If you haven't "fix" your reference name you can do it here but remember you have to replace it in the SAM/BAM header as well. Or go back to the beginning and redo the whole analysis (joke).

In [ ]:
# Replace the reference name
sed -i 's/gi_511533127_gb_CP004011\.1__Treponema_pallidum_subsp\._pallidum_SS14,_complete_genome/gi_511533127_gb_CP004011_1_Treponema_pallidum_subsp_pallidum_SS14_complete_genome/g' $REFERENCE 

# Replace the SAM header
for sample in *.filt.indelRealigned.dedup.MAPQ40.bam
do
    samtools view -@ $THREADS -H $sample > $sample.header.sam  # extract header only

    sed -i 's/gi_511533127_gb_CP004011\.1__Treponema_pallidum_subsp\._pallidum_SS14,_complete_genome/gi_511533127_gb_CP004011_1_Treponema_pallidum_subsp_pallidum_SS14_complete_genome/g' $sample.header.sam

    samtools reheader $sample.header.sam $sample > $sample.tmp
    mv $sample.tmp $sample

    rm $sample.header.sam
done


#### Mapping consensus ~ alignment-guided assembly

In [ ]:
MIN_COVER=3 # All bases bellow this coverage are converted to ns
PLOIDY=1 # Ploidy for the bcftools

mkdir -p $OUTPUT_DIR/consensus/other

# Make sure we have reference index and dictionary (! have to redo if you used the previous "fix" step)
samtools faidx $REFERENCE
rm ${REFERENCE%.*}.dict
picard CreateSequenceDictionary R=$REFERENCE O=${REFERENCE%.*}.dict

for sample in *.filt.indelRealigned.dedup.MAPQ40.bam
do
    samtools index -@ $THREADS $sample
    samtools mpileup --max-depth 10000 -E --min-BQ 13 --fasta-ref $REFERENCE --min-MQ 40 -g -u --max-idepth 100000 --min-ireads 5 --gap-frac 0.002 --excl-flags UNMAP,SECONDARY,QCFAIL,DUP --output-tags DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR --reference $REFERENCE $sample | bcftools call --ploidy $PLOIDY -c --keep-masked-ref --output-type v --threads $THREADS - > $OUTPUT_DIR/consensus/other/${sample%.*}.vcf

    cat $OUTPUT_DIR/consensus/other/${sample%.*}.vcf | vcfutils.pl vcf2fq -d $MIN_COVER | seqtk seq -A - > $OUTPUT_DIR/consensus/${sample%.*}.cns.def.fasta

    # Convert all low-coverage (=lower-case) positions to "n"
    sed -e '/^>/! s/[[:lower:]]/n/g' $OUTPUT_DIR/consensus/${sample%.*}.cns.def.fasta > $OUTPUT_DIR/consensus/${sample%.*}.cns.final.fasta

    gatk -T FastaAlternateReferenceMaker -R $REFERENCE -o $OUTPUT_DIR/consensus/${sample%.*}.cns.gatk.fasta --variant $OUTPUT_DIR/consensus/other/${sample%.*}.vcf
done

### BAM downsampling (optional)

In some cases, our resulting SAM/BAM is very deep which causes problems during the subsequent analysis. An option to lower down the computational demands is to downsample to a specific depth.

In [ ]:
MAX_COVER=250 # Set maximum coverage to put the limit on

for sample in *.filt.indelRealigned.dedup.MAPQ40.bam
do
    java -jar -Xmx4g $CONDA_PREFIX/bin/sortsamrefname.jar --tmpDir $OUTPUT_DIR/tmp $sample |  java -jar -Xmx4g $CONDA_PREFIX/bin/downsamplebam.jar -n $MAX_COVER | samtools sort -@ $THREADS - > ${sample%.bam*}.downsamp${MAX_COVER}.bam
    samtools index -@ $THREADS ${sample%.bam*}.downsamp${MAX_COVER}.bam
done

### Variant call

To get the most relevant variants directly from the alignments we could either use the variants used for the consensus generation or we can use probably more suitable tools for bacteria variant call such as [`freebayes`](https://github.com/ekg/freebayes). The reason why we don't use other tools than `samtools`/`bcftools` for the generation of consensus is the compatibility of the approach. 

In [ ]:
MIN_DP=$MIN_COVER # Set minimal coverage depth of the mapping
QUAL_SET=50 # Set minimal variant quality for the most certain variants; default it 20

mkdir -p $OUTPUT_DIR/variants/freebayes/stats

for sample in *filt.indelRealigned.dedup.MAPQ40.bam
do
    echo "Processing $sample"
    
    samtools index $sample

    freebayes --bam $sample --vcf $OUTPUT_DIR/variants/freebayes/${sample%.*}.full.vcf --fasta-reference $REFERENCE --theta 0.001 --ploidy $PLOIDY --min-mapping-quality 10 --min-base-quality 15 --min-alternate-fraction 0.01 --min-alternate-count 2 --genotype-qualities --report-all-haplotype-alleles # You can add "--report-monomorphic" to get all positions
done

# Variant post-processing and filtering
cd $OUTPUT_DIR/variants/freebayes/

for sample in *full.vcf
do
    echo "Processing " $sample

    # GATK filtering
    gatk -T VariantFiltration -R $REFERENCE -o $OUTPUT_DIR/variants/freebayes/${sample%.full*}.filt.vcf --variant:VCF $sample --filterExpression "DP < ${MIN_DP}" --filterName "LowCoverage" --filterExpression "QUAL > 0 && QUAL < 20" --filterName "VeryLowQual" --filterExpression "QUAL > 20 && QUAL < ${QUAL_SET}" --filterName "LowQual" --filterExpression "QUAL == 0" --filterName "TechnicalQual" # This might be too strict for FreeBayes
    # gatk -T VariantFiltration -R $REFERENCE -o ${sample%.full*}.filt.vcf --variant:VCF $sample --filterExpression "DP < ${MIN_DP}" --filterName "LowCoverage" --filterExpression "QUAL > 0 && QUAL < 20" --filterName "VeryLowQual" --filterExpression "QUAL == 0" --filterName "TechnicalQual" # This would be the way FreeBayes author recommends to use the filtering

    mv ${sample%.full*}.filt.vcf $sample
    rm ${sample%.full*}.filt.vcf.idx
    
    # Get only variants (PASS filtering)
    grep "^#" $sample > ${sample%.full*}.var.vcf # Get only header
    awk '$7 == "PASS" {print $0}' $sample >> ${sample%.full*}.var.vcf

    # Get only SNPs and/or indels
#    vcffilter -f "TYPE = snp" ${sample%.full*}.var.vc > ${sample%.full*}.var.snp.vcf # Get only SNP
#    vcffilter -f "( TYPE = ins | TYPE = del )" ${sample%.full*}.var.vcf > ${sample%.full*}.var.indel.vcf # Get only indels

    # Convert vct to tab vcftools
    cat ${sample%.full*}.var.vcf | vcf-to-tab > ${sample%.full*}.var.tsv

    # Get stats of the variant calling
    # Stats from raw and filtered SNPs
    vcfstats $sample > $OUTPUT_DIR/variants/freebayes/stats/${sample%.vcf}.stats.txt
    vcfstats ${sample%.full*}.var.vcf > $OUTPUT_DIR/variants/freebayes/stats/${sample%.full*}.var.stats.txt

    # Compare ts/tv ration between high/low quality variants
    echo $sample > $OUTPUT_DIR/variants/freebayes/stats/${sample%.vcf}.ratio.stats.txt
    echo "Low quality variants" >> $OUTPUT_DIR/variants/freebayes/stats/${sample%.*}.ratio.stats.txt # Low quality variants
    vcffilter -f "QUAL < 20" $sample | vcfstats | grep "ts\|bial" >> $OUTPUT_DIR/variants/freebayes/stats/${sample%.vcf}.ratio.stats.txt
    echo "High quality variants" >> $OUTPUT_DIR/variants/freebayes/stats/${sample%.vcf}.ratio.stats.txt # High quality variants
    vcffilter -f "QUAL > 20" $sample | vcfstats | grep "ts\|bial" >> $OUTPUT_DIR/variants/freebayes/stats/${sample%.vcf}.ratio.stats.txt
done

### Variant annotation (optional)

Having variants in VCF is nice but often you might be interested what is their effect or purpose. Unless you imidiately know the important positions variant annotation might bring you some additional information. [SnpEff](http://snpeff.sourceforge.net/) allows you to build your own annotation database and apply it to your VCF. 

First, we build the annotation database. This has to be done only once per genome. Note: there are some pre-loaded annotations in SnpEff but since we might be using different genome version it might be safer to generate our own database. You can generate the database with GFF and FASTA (hint [here](http://lab.loman.net/2012/11/16/how-to-get-snpeff-working-with-bacterial-genomes-from-ncbi/)) or you can do it from **GenBank full** format (suffix **.gbff** at the GenBank ftp). The advantage of going for **GenBank full** is that both gene annotation and genome sequence is included in one file.

In [14]:
echo $ANNOT_DIR
echo $seq_name
echo $fasta_header

/mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2/envs/treponema/share/snpeff-4.3.1t-2//data/SS14_CP004011.1
(treponema) 
(treponema) gi_511533127_gb_CP004011_1_Treponema_pallidum_subsp_pallidum_SS14_complete_genome
(treponema) 

: 1

In [17]:
# Get the GenkBank full annotation - reference genome version and the gene annotation must be identical!
#wget ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Treponema_pallidum/all_assembly_versions/GCA_000410555.1_ASM41055v1/GCA_000410555.1_ASM41055v1_genomic.gbff.gz -P $(dirname $REFERENCE)
#gunzip $(dirname $REFERENCE)/GCA_000410555.1_ASM41055v1_genomic.gbff.gz
cd $(dirname $REFERENCE)

DB_NAME="SS14_CP004011.1" # How you want to call the database?
fasta_header=`head -1 $REFERENCE | sed 's/>//g' | cut -d ' ' -f 1` # Get only the fasta header without the initial ">" and split after first empty space

# Create custom snpEff.config
echo "$DB_NAME.genome : Treponema pallidum subsp. pallidum SS14" > ${REFERENCE%.fa*}.snpEffect.config # Database name and description
echo -e "\\t$DB_NAME.chromosomes : $fasta_header" >> ${REFERENCE%.fa*}.snpEffect.config # Database name and fasta header
echo -e "\\t$DB_NAME.$fasta_header.codonTable : Bacterial_and_Plant_Plastid" >> ${REFERENCE%.fa*}.snpEffect.config # Database name, fasta header and codon table

SNPEFF_DIR=$(ls -d $CONDA_PREFIX/share/snpeff-*/)
echo $SNPEFF_DIR
ANNOT_DIR=$SNPEFF_DIR/data/$DB_NAME # Data directory to store the added genome annotation

# Make the annotation dabase - only once per genome!
if [ ! -d "$ANNOT_DIR" ]; then  # Check if the genome folder already exists
    mkdir -p $ANNOT_DIR
    cd $ANNOT_DIR/

    cat $(dirname $REFERENCE)/GCA_000410555.1_ASM41055v1_genomic.gbff > $ANNOT_DIR/genes.gbk

    seq_name=`head -1 $ANNOT_DIR/genes.gbk | awk '{print $2}'` # Cheat a little and replace chromosome id with out sequence name - will work only with a single chromosome reference!!!
    sed -i "s/$seq_name/$fasta_header/g" $ANNOT_DIR/genes.gbk
 
    cat ${REFERENCE%.fa*}.snpEffect.config >> $SNPEFF_DIR/snpEff.config

    cd $SNPEFF_DIR/
    snpEff build -genbank -v $DB_NAME
fi

snpEff dump $DB_NAME | head -50 # Check if everything is OK 

(treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) (treponema) /mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2/envs/treponema/share/snpeff-4.3.1t-2/
(treponema) (treponema) (treponema) (treponema) 00:00:00	SnpEff version SnpEff 4.3t (build 2017-11-24 10:18), by Pablo Cingolani
00:00:00	Command: 'build'
00:00:00	Building database for 'SS14_CP004011.1'
00:00:00	Reading configuration file 'snpEff.config'. Genome: 'SS14_CP004011.1'
00:00:00	Reading config file: /mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2/envs/treponema/share/snpeff-4.3.1t-2/snpEff.config
00:00:00	done
Chromosome: 'gi_511533127_gb_CP004011_1_Treponema_pallidum_subsp_pallidum_SS14_complete_genome'	length: 1139569

	Create exons from CDS (if needed): .......................................................................................................................................

: 1

When we have the database ready we can run the variant effect prediction.

In [18]:
cd $SNPEFF_DIR/

for sample in $OUTPUT_DIR/variants/freebayes/*.vcf
do
    echo $sample
    snpEff $DB_NAME $sample > ${sample%.vcf}.$DB_NAME.annot.vcf
done

(treponema) (treponema) /mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/results/variants/freebayes/CW56_S1_.SS14.filt.indelRealigned.dedup.MAPQ40.full.vcf


NEW VERSION!
	There is a new SnpEff version available: 
		Version      : 4.4
		Release date : 2019-01-26
		Download URL : http://sourceforge.net/projects/snpeff/files/snpEff_latest_core.zip

/mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/results/variants/freebayes/CW56_S1_.SS14.filt.indelRealigned.dedup.MAPQ40.var.vcf


NEW VERSION!
	There is a new SnpEff version available: 
		Version      : 4.4
		Release date : 2019-01-26
		Download URL : http://sourceforge.net/projects/snpeff/files/snpEff_latest_core.zip

(treponema) 

: 1

### *De novo* assembly - mapped reads

To accompany the alignment-guided assembly and to balance for the potentical pitfalls of that approach we strongly recommend to run *de novo* assembly as well. You can run *de novo* assembly on the host-cleaned data but since we assume there could be a mixture of organisms present we recommend to run the assembly only from the initial mapping step ("rough mapping"). 

#### SAM/BAM to fastq

In the preparation step, we first extract the mapped reads from the SAM/BAM.

In [ ]:
mkdir -p $OUTPUT_DIR/alignment/fastq

cd $OUTPUT_DIR/alignment

for sample in *.$(basename $REFERENCE .fa).bam
do
    echo "Processing file $sample"

    samtools view -H $sample > ${sample%.*}.sam

    samtools view -@ $THREADS -F 4 -f 3 $sample  >> ${sample%.*}.sam
    samtools view -@ $THREADS -b ${sample%.*}.sam | samtools sort -@ $THREADS -o ${sample%.*}.mapped.bam -
    rm ${sample%.*}.sam

    picard FixMateInformation I=${sample%.*}.mapped.bam O=tmp.bam VALIDATION_STRINGENCY=LENIENT
    mv tmp.bam ${sample%.*}.mapped.bam
    samtools index -@ $THREADS ${sample%.*}.mapped.bam

    samtools collate -O -@ $THREADS ${sample%.*}.mapped.bam - | samtools fastq -@ $THREADS -s $OUTPUT_DIR/alignment/fastq/${sample%.*}_unpaired.mapped.fastq -1 $OUTPUT_DIR/alignment/fastq/${sample%.*}_R1.mapped.fastq -2 $OUTPUT_DIR/alignment/fastq/${sample%.*}_R2.mapped.fastq -
done

cd $OUTPUT_DIR/alignment/fastq/

rm *_unpaired.mapped.fastq

for sample in *.fastq
do
    pigz -p $THREADS $sample
done

#### *De novo* assembly

Now we can do the *de novo* assembly itself. In this section, we are using SPAdes but you can consider using [Unicycler](https://github.com/rrwick/Unicycler) or [metaSPADES](http://cab.spbu.ru/software/meta-spades/).

In [ ]:
KMERS="15,21,27,33,55,77,99,127" # List of kmers for the SPAdes assembly
NSEQ=10 # How many longest sequences we will report in short assembly fasta result

mkdir -p $OUTPUT_DIR/assembly/spades

for sample in *R1.mapped.fastq.gz
do
    FORWARD=$sample
    extension="${FORWARD##*R1}"
    REVERSE=${FORWARD%R1*}R2${extension}

    # Input file check
    if [ ! -f ${FORWARD} ]; then
        echo "Input file not found! First in pair."; echo ${FORWARD}
    fi
    if [ ! -f ${REVERSE} ]; then
        echo "Input file not found! Second in pair."; echo ${REVERSE}
    fi
    
    NAME=${FORWARD%R1*}

    mkdir $OUTPUT_DIR/assembly/spades/$NAME

    spades.py --careful --threads $THREADS --cov-cutoff 5 --tmp-dir $OUTPUT_DIR/tmp -k $KMERS -1 $FORWARD -2 $REVERSE -o $OUTPUT_DIR/assembly/spades/$NAME
done

# Assembly statistics
cd $OUTPUT_DIR/assembly/spades

dirs_tmp=$(ls -d */)

mkdir scaffolds
mkdir graphs

for sample in $dirs_tmp
do
    cat $sample/scaffolds.fasta > scaffolds/${sample%/*}.scaffolds.fasta
    cat $sample/assembly_graph.fastg > graphs/${sample%/*}.assembly_graph.fastg
    cat $sample/assembly_graph_with_scaffolds.gfa > graphs/${sample%/*}.assembly_graph_with_scaffolds.gfa
done

cd scaffolds/

for sample in *.fasta
do
    awk "/^>/ {n++} n>$NSEQ {exit} {print}" $sample > ${sample%.*}.top${NSEQ}.fasta
done

### Assembly quality check

Once the assembly is done we strongly recommend to run assembly quality check. We also recommend looking at the resulting graphs at [Bandage](https://rrwick.github.io/Bandage/).

In [ ]:
mkdir -p $OUTPUT_DIR/qc/assembly/BUSCO
mkdir $OUTPUT_DIR/qc/assembly/quast

#BUSCO_INI=/storage/brno2/home/opplatek/tools/anaconda3/envs/busco/share/busco-3.0.2-6/conda.config.ini
#export BUSCO_CONFIG_FILE=$SCRATCH/busco.ini

# Get BUSCO core genes annotation database
wget https://busco.ezlab.org/datasets/spirochaetes_odb9.tar.gz -O $OUTPUT_DIR/qc/assembly/BUSCO/spirochaetes_odb9.tar.gz
tar xvzf $OUTPUT_DIR/qc/assembly/BUSCO/spirochaetes_odb9.tar.gz -C $OUTPUT_DIR/qc/assembly/BUSCO

#cp -r /storage/brno2/home/opplatek/tools/anaconda3/pkgs/augustus-3.2.3-boost1.60_0/config $SCRATCH/augustus_config
#export AUGUSTUS_CONFIG_PATH=$SCRATCH/augustus_config

# -sp XXX is one of the supported organisms by Augustus and should be as close as possible to our organism https://github.com/Gaius-Augustus/Augustus/blob/master/docs/RUNNING-AUGUSTUS.md

for sample in *.scaffolds.fasta
do
    mkdir $OUTPUT_DIR/qc/assembly/BUSCO/${sample%.fa*}
    
    run_BUSCO.py -i $sample -o busco_${sample%.fa*} -l $OUTPUT_DIR/qc/assembly/BUSCO/spirochaetes_odb9 -m geno \
    -c $THREADS -sp E_coli_K12 -t $OUTPUT_DIR/tmp
done

mv run_busco* $OUTPUT_DIR/qc/assembly/BUSCO/

quast -r $REFERENCE -t $THREADS --output-dir $OUTPUT_DIR/qc/assembly/quast *.scaffolds.fasta

### Additional scaffolding (optional)

SPAdes is quite efficient in the scaffolding and we never had a need to do additional scaffolding but you might find yourself in a situation to do so. We are using [BESST](https://github.com/ksahlin/BESST) but there are other options such as [SSPACE-basic] (https://github.com/nsoranzo/sspace_basic), [SSPACE-standard](https://www.baseclear.com/services/bioinformatics/basetools/sspace-standard/) and others. 

It is **extremely** important to compare the original assembly scaffolds and the "re-scaffolded" ones and determine which ones we want to use. Longer doesn't always mean better. Carefully check the Quast report and compare the original vs. re-scaffolded and make the decision. I would always check "# misassemblies" and consider if it has improved or not or if there is some biology behind the change. 

For the best functionality, you have know the insert size. You can get this information from the Qualimap report which we created above.

In [27]:
mkdir $OUTPUT_DIR/assembly/spades/scaffolds_besst
mkdir $OUTPUT_DIR/qc/assembly/quast_besst

cd $OUTPUT_DIR/alignment/fastq

for sample in *R1.mapped.fastq.gz
do
    FORWARD=$sample
    extension="${FORWARD##*R1}"
    REVERSE=${FORWARD%R1*}R2${extension}

    # Input file check
    if [ ! -f ${FORWARD} ]; then
        echo "Input file not found! First in pair."; echo ${FORWARD}
    fi
    if [ ! -f ${REVERSE} ]; then
        echo "Input file not found! Second in pair."; echo ${REVERSE}
    fi
    
    contig=$OUTPUT_DIR/assembly/spades/scaffolds/${FORWARD%R1*}.scaffolds.fasta # SPAdes assembly contigs/scaffolds

    bwa index $contig

    bwa mem -t $THREADS -w 0 -O 99 $contig $FORWARD $REVERSE | samtools view -F 4 -@ $THREADS -b - | samtools fixmate -O bam - - | samtools sort -@ $THREADS - > $OUTPUT_DIR/assembly/spades/scaffolds_besst/${FORWARD%R1*}${extension%.fastq.gz*}.bam # Make alignment for BESST; default is BWA-MEM and recommended settings are used; they have script to do it (reads_to_ctg_map.py) but it has some error - setting taken from there

    samtools index -@ $THREADS $OUTPUT_DIR/assembly/spades/scaffolds_besst/${FORWARD%R1*}${extension%.fastq.gz*}.bam

    mkdir $OUTPUT_DIR/assembly/spades/scaffolds_besst/${FORWARD%R1*}

#  -m MEAN [MEAN ...]    Mean insert size of libraries.
#  -s STDDEV [STDDEV ...]
#                        Estimated standard deviation of libraries.
    runBESST -c $contig -f $OUTPUT_DIR/assembly/spades/scaffolds_besst/${FORWARD%R1*}${extension%.fastq.gz*}.bam -filter_contigs 100 -orientation fr -o $OUTPUT_DIR/assembly/spades/scaffolds_besst/${FORWARD%R1*}
done

rm $OUTPUT_DIR/assembly/spades/scaffolds_besst/*.bam*

cd $OUTPUT_DIR/assembly/spades/scaffolds_besst

for sample in $(ls -d */)
do
    cat $sample/BESST_output/pass1/Scaffolds_pass1.fa > $OUTPUT_DIR/assembly/spades/scaffolds_besst/${sample%/*}.scaffolds.fasta
done

cd $OUTPUT_DIR/assembly/spades/scaffolds_besst/

for sample in *.scaffolds.fasta
do
    cat $sample | awk '/^>/ {printf("%s%s\t",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}' | awk -F '\t' '{printf("%d\t%s\n",length($2),$0);}' | sort -k1,1nr | cut -f 2- | tr "\t" "\n" > $sample.tmp # Longest first; shortest first change sort 'sort -k1,1nr' to 'sort -k1,1n'
    mv $sample.tmp $sample
    cat $sample > $OUTPUT_DIR/assembly/spades/scaffolds/${sample%.scaffolds.fasta}.scaffolds_besst.fasta 
done

cd $OUTPUT_DIR/assembly/spades/scaffolds

quast -r $REFERENCE -t $THREADS --output-dir $OUTPUT_DIR/qc/assembly/quast *.scaffolds*.fasta

mkdir: cannot create directory ‘/mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/results/assembly/spades/scaffolds_besst’: File exists
(treponema) (treponema) (treponema) (treponema) [bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.30 seconds elapse.
[bwa_index] Update BWT... 0.01 sec
[bwa_index] Pack forward-only FASTA... 0.01 sec
[bwa_index] Construct SA from BWT and Occ... 0.12 sec
[main] Version: 0.7.15-r1140
[main] CMD: bwa index /mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/results/assembly/spades/scaffolds/CW56_S1_.SS14_.scaffolds.fasta
[main] Real time: 0.452 sec; CPU: 0.448 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 837234 sequences (120000108 bp)...
[M::process] read 837314 sequences (120000224 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (2, 414208, 1, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size di

: 1

### Scaffold contamination scan (optional)

Once we have the assembly checked, ready and potentialy extended using the additional scaffolding we can run the "contamination" scan once more. This time not on the sequencing reads themselves but on the resulting scaffolds. We will be using Kraken2 again but be aware that Kraken2 evaluates the percentages (the first column) based on the number of reads assigned to the individual phylogenetic groups. Since we now have scaffolds of different lengths the statistics might be misleading. It's better to keep only longer scaffolds to avoid counting unscaffolded short sequences which might be heavily over abundant.

In [ ]:
mkdir $OUTPUT_DIR/qc/kraken2/scaffolds

for sample in *.scaffolds.fasta
do
    reformat.sh in=$sample out=$sample.tmp.fasta minlength=1000 # Get only scaffolds >=1000 bp

    kraken2 --threads $THREADS --use-names --classified-out $OUTPUT_DIR/qc/kraken2/scaffolds/${sample%.scaffolds.fasta}.classified-out.fasta --unclassified-out $OUTPUT_DIR/qc/kraken2/scaffolds/${sample%.scaffolds.fasta}.unclassified-out.fasta --output $OUTPUT_DIR/qc/kraken2/scaffolds/${sample%.scaffolds.fasta}.kraken.txt --report $OUTPUT_DIR/qc/kraken2/scaffolds/${sample%.scaffolds.fasta}.kraken.report.txt --db $KRAKEN2_DB $sample.tmp.fasta
    rm $sample.tmp.fasta

    # Extract only the annotated reads and their taxa
    cat $OUTPUT_DIR/qc/kraken2/scaffolds/${sample%.scaffolds.fasta}.kraken.txt | cut -f2,3 > $OUTPUT_DIR/qc/kraken2/scaffolds/${sample%.scaffolds.fasta}.kraken.taxid.txt
done

mkdir: cannot create directory ‘/mnt/ssd/ssd_2/bioda_temp/honza/scratch_default/linda/results/qc/kraken2/scaffolds’: File exists
(treponema) (treponema) java -ea -Xmx200m -cp /mnt/nfs/home/323639/000000-My_Documents/VM-home/tools/miniconda2/envs/treponema/opt/bbmap-38.22-1/current/ jgi.ReformatReads in=CW56_S1_.SS14_.scaffolds.fasta out=CW56_S1_.SS14_.scaffolds.fasta.tmp.fasta minlength=1000
Executing jgi.ReformatReads [in=CW56_S1_.SS14_.scaffolds.fasta, out=CW56_S1_.SS14_.scaffolds.fasta.tmp.fasta, minlength=1000]

Input is being processed as unpaired
Input:                  	67 reads          	1153760 bases
Short Read Discards:    	48 reads (71.64%) 	20126 bases (1.74%)
Output:                 	19 reads (28.36%) 	1133634 bases (98.26%)

Time:                         	0.381 seconds.
Reads Processed:          67 	0.18k reads/sec
Bases Processed:       1153k 	3.03m bases/sec


### Gene prediction (optional)

If we are sure out assembly is correct, clean and we want to proceed to further analysis we can try gene prediction on the scaffolded assembly. We use [PROKKA](https://github.com/tseemann/prokka) but there are other options as well such as [RAST](http://rast.nmpdr.org/), [Augustus](http://bioinf.uni-greifswald.de/webaugustus/index.gsp), [funannotate](https://github.com/nextgenusfs/funannotate).

In [ ]:
# If we have a set of HIGHLY trusted proteins we can add the in genebank (gbk) format and assign GBK variable pointing to the file
#GBK=/path/to/species_trusted_proteins.gbk

for sample in *.scaffolds.fasta
do
    prokka --cpus $THREADS --kingdom Bacteria --outdir $OUTPUT_DIR/assembly/gene_prediction --prefix ${sample%.fa*} $sample # --proteins $GBK
done

At this point, there is bug in PROKKA-minced conda installation. If you get an error with `Error: A JNI error has occurred, please check your installation and try again
Exception in thread "main" java.lang.UnsupportedClassVersionError: minced has been compiled by a more recent version of the Java Runtime (class file version 55.0), this version of the Java Runtime only recognizes class file versions up to 52.0...` you have to downgrade minced to version 0.3.3 (or rebuild from souce). Most likely right now it's at version 0.4.0 and Java at version 1.8.0. 

In [ ]:
# Results in error
minced --version
# Check minced version
ls -l $CONDA_PREFIX/bin/minced
# Check java version
java -version
# Download minced-0.3.0
mkdir $CONDA_PREFIX/share/minced-0.3.3
wget https://github.com/ctSkennerton/minced/releases/download/0.3.3/minced.jar -O $CONDA_PREFIX/share/minced-0.3.3/minced.jar
wget https://github.com/ctSkennerton/minced/releases/download/0.3.3/minced -O $CONDA_PREFIX/share/minced-0.3.3/minced
chmod u+x $CONDA_PREFIX/share/minced-0.3.3/minced
export PATH=$CONDA_PREFIX/share/minced-0.3.3:$PATH
# Should work fine
which minced
minced --version

And now you can rerun the gene prediction (PROKKA).

### Whole genome alignment (optional)

Another thing we can do with the assembly is to look at the whole-genome similarities and try to fix the orientation of the resulting scaffolds. This can be done using whole-genome alignment of the scaffolds and the related reference.

In [ ]:
mkdir $OUTPUT_DIR/assembly/genome_alignment

for sample in *.scaffolds.fasta
do
    sample_name=$(basename $sample .fasta)

    mkdir $OUTPUT_DIR/assembly/genome_alignment/$sample_name

    nucmer -p $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer $REFERENCE $sample

    dnadiff -p $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer -d $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.delta

    mummerplot -p $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer -s large --SNP --postscript --filter $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.delta
    mummerplot -p $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer -s large --SNP --png --filter $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.delta

    mkdir $OUTPUT_DIR/assembly/genome_alignment/$sample_name/individual_alignments

    grep ">" $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.delta | cut -d ' ' -f1,2 | sed 's/>//g' | while read list
    do
        echo $list
        outname=`echo $list | sed 's/ /_/g'`
        
        show-aligns $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.delta $list > $OUTPUT_DIR/assembly/genome_alignment/$sample_name/individual_alignments/$outname.aln
    done

    # Extract unaligned scaffolds to the reference
    cat $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.unqry | cut -f1 > $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.unqry.names
    seqtk subseq $sample $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.unqry.names > $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.unqry.fasta
    rm $OUTPUT_DIR/assembly/genome_alignment/$sample_name/nucmer.unqry.names
done

### Outputs